In [1]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
import time as time

In [2]:
pdb=app.PDBFile('TiO21aHigher1.pdb')
forcefield=mm.app.ForceField('TiO2_imcomplete.xml')


C:\Users\Garrett\Anaconda3\lib\site-packages\simtk\openmm\app\internal\pdbstructure.py:536: UserWarning: WARNING: duplicate atom (HETATM    3  O           0       0.000  -1.362  -0.573  1.00  0.00           O  , HETATM    2  O           0       0.000   1.362  -0.573  1.00  0.00           O  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))


In [3]:
system=forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff)
integrator=mm.LangevinIntegrator(208.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

platform=mm.Platform.getPlatformByName('Reference')
simulation=app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

In [4]:
print('minimizing...')

st=simulation.context.getState(getPositions=True,getEnergy=True)
print('PE b4 mim is %s' %(st.getPotentialEnergy()))

simulation.minimizeEnergy(maxIterations=100)

st=simulation.context.getState(getPositions=True,getEnergy=True)
print("PE after mim 4 running is %s" %(st.getPotentialEnergy()))

minimizing...
PE b4 mim is 0.00043470359749896815 kJ/mol
PE after mim 4 running is 2.4199383258342844e-05 kJ/mol


In [5]:
print('Equibiration...')

simulation.reporters.append(app.StateDataReporter(stdout, 250, step=True,
                                                 potentialEnergy=True, temperature=True, separator='\t'))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(5000)

Equibiration...
#"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"
250	1.1130051643462964	204.61730041862575
500	3.460209128288848	377.87542098202005
750	4.236584636356106	82.91150467818012
1000	2.336725719686399	140.47220519881216
1250	2.712477038681424	115.31337175315896
1500	0.6523936435900981	140.74247636648607
1750	0.44671174576309236	172.03864268481286
2000	2.178302409744627	328.998929608853
2250	0.5786938903057374	138.82191592238257
2500	2.3075984243868457	266.21168564061145
2750	2.9283648571322702	293.14716397107367
3000	2.6136019474841796	117.87349613295605
3250	3.135815799052192	170.79793125789746
3500	0.8409820332843615	137.9503928967031
3750	3.9569838203513217	430.4839900060991
4000	6.857562558979872	119.21549251482472
4250	3.2482063584422445	299.3947014193171
4500	0.4173208810767621	86.1538743987665
4750	2.0942231240678653	107.36673193514925
5000	1.4278862092940956	443.41345141565967


In [6]:
tinit=time.time()
simulation.reporters.clear()
simulation.reporters.append(app.StateDataReporter(stdout,50000, 
                                                  step=True, 
                                                  time=True, 
                                                  potentialEnergy=True, 
                                                  temperature=True, 
                                                 speed=True, 
                                                 separator='\t'))
simulation.reporters.append(app.DCDReporter('TiO21a.dcd', 100))

simulation.step(20000000)

tfinal=time.time()

print('Done!')
print('Time required for simualtion:', tfinal-tinit, 'seconds')

#"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"
50000	99.99999999994834	1.7480516009391054	156.37433652313604	--
100000	200.00000000022686	1.379503337980444	173.18492117620767	8.11e+04
150000	300.00000000070435	1.1570402874443553	62.036947533173134	8.54e+04
200000	400.00000000118183	5.0227355871276975	105.6303214263374	8.13e+04
250000	500.0000000016593	0.679728902434052	75.67333595050543	8.15e+04
300000	599.9999999996356	3.532837792054668	126.6545712356846	8.32e+04
350000	699.999999997271	1.895820420796686	246.77831909535303	8.13e+04
400000	799.9999999949063	0.860006480989468	246.72110988346498	8.04e+04
450000	899.9999999925416	0.38892002000083226	147.3001667222311	7.88e+04
500000	999.9999999901769	5.856793827743233	166.31897815294096	7.89e+04
550000	1099.9999999878123	2.7660268868502005	88.9915846329039	7.9e+04
600000	1199.9999999854476	3.3803857343425405	305.23777641208704	7.81e+04
650000	1299.999999983083	4.270392092677191	181.79042084996027	7.86e

5700000	11400.000001404986	3.121907065346498	119.94141462222193	7.3e+04
5750000	11500.000001425358	4.9037989476195785	286.48028700650065	7.31e+04
5800000	11600.000001445731	0.23470839115242909	285.4452330622114	7.32e+04
5850000	11700.000001466104	1.9078237727248086	69.50267907428085	7.32e+04
5900000	11800.000001486476	1.3781017027869318	186.70941777357262	7.33e+04
5950000	11900.000001506849	4.43260879752924	165.86595296254507	7.34e+04
6000000	12000.000001527222	3.9082237091920238	196.18577409946502	7.35e+04
6050000	12100.000001547594	0.4587109062268273	403.2696838625562	7.35e+04
6100000	12200.000001567967	0.7094577220109208	149.35070121648602	7.36e+04
6150000	12300.00000158834	3.2590631506306	121.3389527483918	7.36e+04
6200000	12400.000001608712	1.8288427779762513	235.8963545177723	7.37e+04
6250000	12500.000001629085	3.836273291723065	76.77594877833333	7.37e+04
6300000	12600.000001649458	1.6224220758621837	130.22059892258403	7.38e+04
6350000	12700.00000166983	4.469072741447435	109.0781

11350000	22700.000003707097	5.431366548364938	338.4001543757307	7.65e+04
11400000	22800.00000372747	0.9882349141395845	424.61630966393454	7.65e+04
11450000	22900.000003747842	0.6970553133772077	123.62068256089931	7.66e+04
11500000	23000.000003768215	0.92506884441633	206.62611721305166	7.65e+04
11550000	23100.000003788587	3.816337082993701	215.6098803678008	7.65e+04
11600000	23200.00000380896	4.303912672005515	242.7901993764171	7.66e+04
11650000	23300.000003829333	0.1301721071658011	44.97938747565264	7.66e+04
11700000	23400.000003849706	4.099967106472686	169.461460700467	7.66e+04
11750000	23500.00000387008	4.272606064121151	366.3933170280937	7.67e+04
11800000	23600.00000389045	4.48030713017032	297.3670082938139	7.67e+04
11850000	23700.000003910824	0.5953289287452501	228.0854639670693	7.67e+04
11900000	23800.000003931196	6.545389994397874	198.62290459653508	7.67e+04
11950000	23900.00000395157	1.6472706353849427	130.3580729584886	7.68e+04
12000000	24000.00000397194	2.089249523011385	71.33

16950000	33900.00000598883	3.4939878190843108	193.48953075447213	7.75e+04
17000000	34000.000006009206	0.9982758888651919	84.91829677707241	7.75e+04
17050000	34100.00000602958	0.2092577793748341	134.75863193429365	7.76e+04
17100000	34200.00000604995	0.45947370284998595	281.32695288336686	7.76e+04
17150000	34300.000006070324	1.490336296707395	240.12587240016992	7.76e+04
17200000	34400.0000060907	0.854931183139467	402.5635165116374	7.76e+04
17250000	34500.00000611107	2.4798172431148378	128.15228409934272	7.76e+04
17300000	34600.00000613144	5.631598964150244	257.5759686613157	7.76e+04
17350000	34700.000006151815	6.1750212485963125	17.30506161391723	7.76e+04
17400000	34800.00000617219	2.1110900235274657	330.95386082252566	7.76e+04
17450000	34900.00000619256	8.123530390843824	133.0909808492268	7.76e+04
17500000	35000.00000621293	5.492963534087878	246.10369129468407	7.76e+04
17550000	35100.000006233306	0.4837217711434089	28.369949887239855	7.76e+04
17600000	35200.00000625368	3.988278841813776